In [11]:
# Gerekli kütüphaneleri import edelim
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV, cross_val_score, KFold
from sklearn.metrics import mean_squared_error, make_scorer
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
import lightgbm as lgb

import warnings
warnings.filterwarnings('ignore')

# 1. Veri Setini Yükleme
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test_x.csv')

# 2. Veri Ön İşleme ve Temizleme

# a. Yaş hesaplama ve 'Yas' sütununu ekleme
def calculate_age(born):
    try:
        born = pd.to_datetime(born, errors='coerce', infer_datetime_format=True)
        today = pd.to_datetime('today')
        age = today.year - born.year - ((today.month, today.day) < (born.month, born.day))
        return int(age)
    except:
        return np.nan

# Yaşları hesaplayalım
train_df['Yas'] = train_df['Dogum Tarihi'].apply(calculate_age)
test_df['Yas'] = test_df['Dogum Tarihi'].apply(calculate_age)

# b. Gereksiz sütunları çıkarma
columns_to_drop = ['Dogum Tarihi', 'Dogum Yeri', 'Lise Adi', 'Lise Adi Diger', 'Lise Sehir',
                   'Lise Bolum Diger', 'Lise Bolumu', 'Daha Önceden Mezun Olunduysa, Mezun Olunan Üniversite']
train_df.drop(columns=columns_to_drop, inplace=True)
test_df.drop(columns=columns_to_drop, inplace=True)

# c. Binary sütunların kodlanması
binary_columns = [
    'Burs Aliyor mu?', 'Daha Once Baska Bir Universiteden Mezun Olmus',
    'Baska Bir Kurumdan Burs Aliyor mu?', 'Girisimcilik Kulupleri Tarzi Bir Kulube Uye misiniz?',
    'Profesyonel Bir Spor Daliyla Mesgul musunuz?', 'Aktif olarak bir STK üyesi misiniz?',
    'Stk Projesine Katildiniz Mi?', 'Girisimcilikle Ilgili Deneyiminiz Var Mi?', 'Ingilizce Biliyor musunuz?'
]

for col in binary_columns:
    train_df[col] = train_df[col].map({'Evet': 1, 'Hayır': 0})
    test_df[col] = test_df[col].map({'Evet': 1, 'Hayır': 0})

# 'Cinsiyet' sütunu için kodlama
train_df['Cinsiyet'] = train_df['Cinsiyet'].map({'Erkek': 2, 'Kadın': 1, 'ERKEK': 2, 'KADIN': 1, 'Belirtmek istemiyorum': 0})
test_df['Cinsiyet'] = test_df['Cinsiyet'].map({'Erkek': 2, 'Kadın': 1, 'ERKEK': 2, 'KADIN': 1, 'Belirtmek istemiyorum': 0})

# 'Universite Turu' sütunu için kodlama
train_df['Universite Turu'] = train_df['Universite Turu'].map({'Özel': 1, 'Devlet': 0})
test_df['Universite Turu'] = test_df['Universite Turu'].map({'Özel': 1, 'Devlet': 0})

# d. Kategorik değişkenlerin kodlanması
categorical_columns = train_df.select_dtypes(include=['object']).columns

# Ortak sütunlar için birleşik veri seti oluşturma (Label Encoding için)
combined_df = pd.concat([train_df[categorical_columns], test_df[categorical_columns]], axis=0)

le = LabelEncoder()
for col in categorical_columns:
    combined_df[col] = combined_df[col].astype(str)
    combined_df[col] = le.fit_transform(combined_df[col])

# Tekrar ayırma
train_df[categorical_columns] = combined_df.iloc[:len(train_df)][categorical_columns]
test_df[categorical_columns] = combined_df.iloc[len(train_df):][categorical_columns]

# e. Eksik değerlerin doldurulması
# Eğitim verisindeki eksik değerleri dolduralım
for col in train_df.columns:
    if train_df[col].isnull().sum() > 0:
        if train_df[col].dtype in ['float64', 'int64']:
            median = train_df[col].median()
            train_df[col].fillna(median, inplace=True)
        else:
            mode = train_df[col].mode()[0]
            train_df[col].fillna(mode, inplace=True)

# Test verisindeki eksik değerleri eğitim verisiyle dolduralım
for col in test_df.columns:
    if test_df[col].isnull().sum() > 0:
        if test_df[col].dtype in ['float64', 'int64']:
            median = train_df[col].median()  # Eğitim setindeki medyanı kullan
            test_df[col].fillna(median, inplace=True)
        else:
            mode = train_df[col].mode()[0]  # Eğitim setindeki en sık değeri kullan
            test_df[col].fillna(mode, inplace=True)

# 3. Özellik Mühendisliği

# a. Yeni özellikler
train_df['Ebeveyn Egitim Ort'] = (train_df['Anne Egitim Durumu'] + train_df['Baba Egitim Durumu']) / 2
test_df['Ebeveyn Egitim Ort'] = (test_df['Anne Egitim Durumu'] + test_df['Baba Egitim Durumu']) / 2

# 4. Modelleme

# a. Hedef değişkeni ve özellikleri ayırma
X = train_df.drop(['Degerlendirme Puani', 'id'], axis=1)
y = train_df['Degerlendirme Puani']

# b. Eğitim ve test setlerinin sütunlarını eşleştirme
test_features = test_df.drop('id', axis=1)
common_cols = X.columns.intersection(test_features.columns)
X = X[common_cols]
test_features = test_features[common_cols]

# c. Cross-Validation ve Model Seçimi

# Kullanılacak modeller ve hiperparametreler
models = {
    'XGBoost': {
        'model': xgb.XGBRegressor(objective='reg:squarederror', random_state=42),
        'params': {
            'n_estimators': [100, 200],
            'max_depth': [3, 5, 7],
            'learning_rate': [0.01, 0.1],
            'subsample': [0.8, 1]
        }
    },
    'LightGBM': {
        'model': lgb.LGBMRegressor(random_state=42),
        'params': {
            'n_estimators': [100, 200],
            'max_depth': [-1, 10, 20],
            'learning_rate': [0.01, 0.1],
            'num_leaves': [31, 50]
        }
    }
}

# Performans sonuçlarını saklamak için
model_scores = []

# K-Fold Cross-Validation
kf = KFold(n_splits=5, shuffle=True, random_state=42)

for model_name, mp in models.items():
    print(f"{model_name} modeli için eğitim yapılıyor...")
    grid_search = GridSearchCV(
        estimator=mp['model'],
        param_grid=mp['params'],
        cv=kf,
        scoring='neg_mean_squared_error',
        n_jobs=1,  # Paralel işleme kapatıldı
        verbose=1
    )
    grid_search.fit(X, y)
    best_rmse = np.sqrt(-grid_search.best_score_)
    print(f"{model_name} için en iyi RMSE: {best_rmse}")
    print(f"En iyi hiperparametreler: {grid_search.best_params_}\n")
    model_scores.append({
        'Model': model_name,
        'Best_RMSE': best_rmse,
        'Best_Params': grid_search.best_params_,
        'Best_Estimator': grid_search.best_estimator_
    })

# Sonuçları DataFrame olarak saklayalım
results_df = pd.DataFrame(model_scores).sort_values(by='Best_RMSE')
print("Model Karşılaştırma Sonuçları:")
print(results_df[['Model', 'Best_RMSE']])

# En iyi modeli seçelim
best_model_info = results_df.iloc[0]
best_model = best_model_info['Best_Estimator']
print(f"\nEn iyi model: {best_model_info['Model']}")

# 5. En İyi Model ile Tahmin ve Sonuç Dosyasının Oluşturulması

# Tüm veri üzerinde (X, y) en iyi modeli eğitelim
best_model.fit(X, y)

# Test verisi üzerinde tahmin yapalım
test_predictions = best_model.predict(test_features)

# Negatif tahminleri 0'a eşitleyelim (Eğer gerekiyorsa)
test_predictions = np.where(test_predictions < 0, 0, test_predictions)

submission = pd.DataFrame({
    'id': test_df['id'],
    'Degerlendirme Puani': test_predictions
})

submission.to_csv('submission.csv', index=False)
print("\nTahminler 'submission.csv' dosyasına kaydedildi.")


XGBoost modeli için eğitim yapılıyor...
Fitting 5 folds for each of 24 candidates, totalling 120 fits
XGBoost için en iyi RMSE: 5.569376182089882
En iyi hiperparametreler: {'learning_rate': 0.1, 'max_depth': 7, 'n_estimators': 200, 'subsample': 1}

LightGBM modeli için eğitim yapılıyor...
Fitting 5 folds for each of 24 candidates, totalling 120 fits
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002561 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2215
[LightGBM] [Info] Number of data points in the train set: 52100, number of used features: 36
[LightGBM] [Info] Start training from score 32.095374
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing